In [ ]:
###########################
# data libraries
###########################
import pandas as pd
import numpy as np

###########################
# plot libraries
###########################
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams


###########################
# data generation
###########################
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

### Import and Function
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)

###########################
# transform classes into numbers
###########################
def categoricalToNumerical(df, cols):
  enc = OrdinalEncoder(dtype=np.int16)
  df[cols]= enc.fit_transform(df[cols])


In [10]:
train = pd.read_csv("new_train_v5.csv")
test = pd.read_csv("test_v4.csv")

enc = OrdinalEncoder(dtype=np.int16)
train[['Year', 'Month','Consumption' ,'Consumer_number', 'Installation_zone']]= enc.fit_transform(train[['Year', 'Month','Consumption' ,'Consumer_number', 'Installation_zone']])

enc_target = OrdinalEncoder(dtype=np.int16)
train[['Consumer_type']] = enc_target.fit_transform(train[['Consumer_type']])

train_target = train['Consumer_type']
train.drop(columns=['Consumer_type'],inplace=True)
train.drop(columns=['index'],inplace=True)

rfc = RandomForestClassifier(
    ccp_alpha=0,
    n_jobs=-1,
    n_estimators=10,
    max_depth=10,
    random_state=42,
    oob_score=True,
    max_features=None,
    bootstrap=True,
)

rfc.fit(train, train_target)
rfc.score(train, train_target)

c:\Users\marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:578: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


0.9255597190724261

In [39]:
competition = pd.read_csv("datasets/competition.csv")
categoricalToNumerical(competition, ['Year', 'Month','Consumption' ,'Consumer_number', 'Installation_zone'])

prediction = rfc.predict(competition)
prediction = pd.DataFrame(prediction, columns=['Consumer_type'])
competition = enc.inverse_transform(competition)
prediction = enc_target.inverse_transform(prediction)
competition = pd.DataFrame(competition, columns=['Year', 'Month','Consumption' ,'Consumer_number', 'Installation_zone'])

In [45]:
prediction_df = pd.DataFrame(prediction, columns=['Consumer_type'])
competition['Consumer_type'] = prediction_df['Consumer_type']
competition = competition[['Consumer_number', 'Consumer_type']]
competition.to_csv('competition_v1.csv', index=False)